In [60]:
import pandas as pd
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [67]:
#Createing the connection to the database part 1
mongo = MongoClient(port=27017)
db = mongo['hockey_model']
db.list_collection_names()

['2022-2023', '2023-2023']

In [68]:
#creating connections part 2: putting it into a dataframe
data_2022_2023 = db['2022-2023']
answer = list(data_2022_2023.find({}))
df_2022 = pd.json_normalize(answer)

In [69]:
df_2022.head()

,_id,,playerId,season,name,team,position,situation,games_played,icetime,...,OnIce_A_scoreAdjustedShotsAttempts,OnIce_A_unblockedShotAttempts,OnIce_A_scoreAdjustedUnblockedShotAttempts,OnIce_A_xGoalsFromxReboundsOfShots,OnIce_A_xGoalsFromActualReboundsOfShots,OnIce_A_reboundxGoals,OnIce_A_xGoals_with_earned_rebounds,OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted,Line_or_Pair
0,659df24b1824a1bb36f2b0aa,0,8471817,2022,Ryan Reaves,MIN,R,all,73,40825,...,674.73,491,489.13,4.67,6.20,6.20,27.06,26.92,26.43,3rd line
1,659df24b1824a1bb36f2b0ab,1,8480950,2022,Ilya Lyubushkin,BUF,D,all,68,61236,...,1104.67,863,864.75,10.65,15.01,14.72,62.63,62.82,60.81,3rd pair
2,659df24b1824a1bb36f2b0ac,2,8475625,2022,Matt Irwin,WSH,D,all,61,47720,...,791.60,562,565.64,5.69,5.62,5.62,34.16,34.30,33.73,3rd pair
3,659df24b1824a1bb36f2b0ad,3,8480860,2022,Kevin Bahl,NJD,D,all,42,35331,...,508.74,378,379.33,3.76,7.03,7.03,21.02,21.10,20.84,3rd pair
4,659df24b1824a1bb36f2b0ae,4,8477952,2022,Robby Fabbri,DET,C,all,28,26877,...,425.03,316,313.93,3.19,2.47,2.47,20.77,20.86,20.65,2nd line


In [70]:
df_2022.drop(['_id','playerId','season','name','team','situation'], axis='columns',inplace=True)
df_2022.head()

,,position,games_played,icetime,minutes/game,onIce_xGoalsPercentage,onIce_corsiPercentage,onIce_fenwickPercentage,iceTimeRank,I_F_xOnGoal,...,OnIce_A_scoreAdjustedShotsAttempts,OnIce_A_unblockedShotAttempts,OnIce_A_scoreAdjustedUnblockedShotAttempts,OnIce_A_xGoalsFromxReboundsOfShots,OnIce_A_xGoalsFromActualReboundsOfShots,OnIce_A_reboundxGoals,OnIce_A_xGoals_with_earned_rebounds,OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted,Line_or_Pair
0,0,R,73,40825,9.32,0.45,0.45,0.46,814,49.47,...,674.73,491,489.13,4.67,6.20,6.20,27.06,26.92,26.43,3rd line
1,1,D,68,61236,15.01,0.39,0.42,0.41,329,37.10,...,1104.67,863,864.75,10.65,15.01,14.72,62.63,62.82,60.81,3rd pair
2,2,D,61,47720,13.04,0.44,0.45,0.46,338,69.79,...,791.60,562,565.64,5.69,5.62,5.62,34.16,34.30,33.73,3rd pair
3,3,D,42,35331,14.02,0.56,0.53,0.52,242,32.17,...,508.74,378,379.33,3.76,7.03,7.03,21.02,21.10,20.84,3rd pair
4,4,C,28,26877,16.00,0.52,0.46,0.46,139,39.69,...,425.03,316,313.93,3.19,2.47,2.47,20.77,20.86,20.65,2nd line


In [ ]:
df_2022.to_csv('model_2022-2023.csv')

In [73]:
# Apply get_dummies on a single column
dummies = pd.get_dummies(df_2022['position'])

# Drop the column that get_dummies was used on
df_without_column = df.drop('D', axis=1)

# Concatenate the original DataFrame with the dummies DataFrame
df_2022_dummies = pd.concat([df_2022, dummies], axis=1)
df_2022_dummies.head()

,,position,games_played,icetime,minutes/game,onIce_xGoalsPercentage,onIce_corsiPercentage,onIce_fenwickPercentage,iceTimeRank,I_F_xOnGoal,...,OnIce_A_xGoalsFromActualReboundsOfShots,OnIce_A_reboundxGoals,OnIce_A_xGoals_with_earned_rebounds,OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted,Line_or_Pair,C,D,L,R
0,0,R,73,40825,9.32,0.45,0.45,0.46,814,49.47,...,6.20,6.20,27.06,26.92,26.43,3rd line,0,0,0,1
1,1,D,68,61236,15.01,0.39,0.42,0.41,329,37.10,...,15.01,14.72,62.63,62.82,60.81,3rd pair,0,1,0,0
2,2,D,61,47720,13.04,0.44,0.45,0.46,338,69.79,...,5.62,5.62,34.16,34.30,33.73,3rd pair,0,1,0,0
3,3,D,42,35331,14.02,0.56,0.53,0.52,242,32.17,...,7.03,7.03,21.02,21.10,20.84,3rd pair,0,1,0,0
4,4,C,28,26877,16.00,0.52,0.46,0.46,139,39.69,...,2.47,2.47,20.77,20.86,20.65,2nd line,1,0,0,0


In [66]:
#Splitting the data
X = df_2022_dummies.copy()
X.drop('I_F_points', axis=1, inplace=True)
y = df_2022_dummies['I_F_points'].ravel()

In [57]:
#further splitting into test and train data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=12)

In [58]:
#creating the scaler variable
scaler = StandardScaler()

In [59]:
X_scaler = scaler.fit(X_train)

ValueError: could not convert string to float: 'R'